In [127]:
from pyspark import SparkContext
from operator import add

In [ ]:
sc = SparkContext(appName="flight")

In [128]:
flights_lines = sc.textFile('flights.csv.bz2')
flights_lines = flights_lines.zipWithIndex().filter(lambda kv: kv[1] > 2).keys()
flights_raw = flights_lines.map(lambda x: x.split(','))
flights = flights_raw.map(lambda p: (p[0], p[1] , p[2] , p[3], p[4] , p[5] , p[6], p[7] , p[8] , p[9], p[10], '0' if p[11] == '' else p[11], p[24] ))

airports_lines = sc.textFile('airports.csv')
airports_lines = airports_lines.zipWithIndex().filter(lambda kv: kv[1] > 2).keys()
airports = airports_lines.map(lambda x: x.split(','))

In [129]:
# Question 1: Find a list of all origin Airports. Store the list of all origin airports in a single file
originAirports = flights.map(lambda x: x[7]).distinct()
originAirports.collect()

['SLC',
 'PSE',
 'ONT',
 'DFW',
 'BJI',
 'PHL',
 'BUF',
 'MFR',
 'IDA',
 'SBA',
 'BDL',
 'OMA',
 'LAN',
 'PBI',
 'OAK',
 'MEM',
 'OAJ',
 'JLN',
 'RDU',
 'CAE',
 'TXK',
 'DLH',
 'INL',
 'SPI',
 'CPR',
 'PAH',
 'ESC',
 'MFE',
 'SBP',
 'DRO',
 'TWF',
 'BPT',
 'ALB',
 'CMI',
 'GTF',
 'HSV',
 'BUR',
 'LBE',
 'GUM',
 'ALO',
 'LCH',
 'MEI',
 'GCK',
 'DIK',
 'AVP',
 'ICT',
 'FLG',
 'EYW',
 'CRW',
 'MQT',
 'ECP',
 'GFK',
 'TOL',
 'BIL',
 'STX',
 'SCC',
 'FNT',
 'MMH',
 'APN',
 'WRG',
 'VLD',
 'SIT',
 'BQK',
 'CSG',
 'UST',
 'BGR',
 'WYS',
 'DLG',
 'GST',
 '14747',
 '12892',
 '14107',
 '10141',
 '14683',
 '10721',
 '12191',
 '14783',
 '10423',
 '14842',
 '11057',
 '11447',
 '12891',
 '10994',
 '10792',
 '14492',
 '14960',
 '11995',
 '10713',
 '11471',
 '12478',
 '11066',
 '11140',
 '13277',
 '11697',
 '14952',
 '10408',
 '11423',
 '14027',
 '13476',
 '10529',
 '10561',
 '12197',
 '13795',
 '13029',
 '13422',
 '11980',
 '10158',
 '11109',
 '11525',
 '11648',
 '12758',
 '13830',
 '15412',
 '11337'

In [130]:
# Question 2: Find a list of (Origin, Destination) pairs
Origin_Destination = flights.map(lambda x: (x[7],x[8])).distinct()
Origin_Destination.collect()

[('SEA', 'ANC'),
 ('LAS', 'ATL'),
 ('ANC', 'SEA'),
 ('ANC', 'PDX'),
 ('PDX', 'MSP'),
 ('SJU', 'MCO'),
 ('HNL', 'KOA'),
 ('IAH', 'MIA'),
 ('HNL', 'LIH'),
 ('SEA', 'DEN'),
 ('ORD', 'FLL'),
 ('RDD', 'SFO'),
 ('BUF', 'JFK'),
 ('ORD', 'DTW'),
 ('SMF', 'DEN'),
 ('AUS', 'LAX'),
 ('BOS', 'BWI'),
 ('PHX', 'DTW'),
 ('MFR', 'DEN'),
 ('SEA', 'IAH'),
 ('BOS', 'PBI'),
 ('BOS', 'JFK'),
 ('JFK', 'BQN'),
 ('CHS', 'EWR'),
 ('CHS', 'DFW'),
 ('BOS', 'BUF'),
 ('PDX', 'EWR'),
 ('IND', 'PHX'),
 ('BOS', 'EWR'),
 ('MCI', 'PHX'),
 ('MCI', 'IAH'),
 ('BDL', 'IAD'),
 ('RDM', 'DEN'),
 ('CID', 'ATL'),
 ('MCI', 'ORD'),
 ('ORD', 'LGA'),
 ('OAK', 'PHX'),
 ('PIT', 'IAH'),
 ('PHX', 'SLC'),
 ('SFO', 'LAX'),
 ('LAX', 'ORD'),
 ('IAH', 'PHX'),
 ('BWI', 'MIA'),
 ('TPA', 'DFW'),
 ('FAI', 'ANC'),
 ('PDX', 'SJC'),
 ('PBI', 'JFK'),
 ('AUS', 'JFK'),
 ('DCA', 'BOS'),
 ('LGA', 'DTW'),
 ('RNO', 'SLC'),
 ('ORD', 'ATL'),
 ('MEM', 'EWR'),
 ('TYS', 'IAD'),
 ('ABE', 'DTW'),
 ('ERI', 'ORD'),
 ('ORD', 'MIA'),
 ('ATL', 'TTN'),
 ('CLL', 'DFW'

In [131]:
# Question 3: Which airport had the largest departure delay in January?
LargestDepartureDelay_January = flights.filter(lambda x: x[1] == "1").map(lambda x: (x[7],float(x[11]))).top(1, lambda x:x[1])
LargestDepartureDelay_January

[('BHM', 1988.0)]

In [132]:
# Question 4: Which airline carrier had the largest delay on weekends (Saturdays and Sundays)?
LargestDelay_Weekends_airline = flights.filter(lambda x: x[3] == "6" or x[3] == "7").map(lambda x: (x[4],float(x[11]))).top(1, lambda x: x[1])
LargestDelay_Weekends_airline

[('AA', 1878.0)]

In [133]:
# Question 5: Which airport has the most cancellation of flights?
HighCancellation_Airport = flights.filter(lambda x: x[12] == "1").map(lambda x: (x[7],1)).reduceByKey(add).top(1, lambda x: x[1])
HighCancellation_Airport


[('ORD', 8548)]

In [134]:
# Question 6: What are the flights cancellation percentage ratio for each carrier? Provide a printout
total_flight = flights.map(lambda x: (x[4],1)).reduceByKey(add)
cancelled_fligth = flights.filter(lambda x: x[12]=="1").map(lambda x: (x[4],1)).reduceByKey(add)
total_cancelled = total_flight.leftOuterJoin(cancelled_fligth)
carrier_cancellation = total_cancelled.map(lambda x: (x[0],(x[1][0],0)) if x[1][1] is None else (x[0],x[1])).map(lambda x: (x[0],x[1][1]/x[1][0]))
carrier_cancellation.collect()

[('AA', 0.015040297086846387),
 ('NK', 0.017072900604026275),
 ('UA', 0.01274521400053905),
 ('B6', 0.016012102693148796),
 ('MQ', 0.05099581851258519),
 ('F9', 0.006473204456382932),
 ('WN', 0.012713822111098344),
 ('OO', 0.01692861258462182),
 ('EV', 0.026628693111785964),
 ('US', 0.020466497244797825),
 ('DL', 0.004365889886868193),
 ('HA', 0.0022419760855884206),
 ('AS', 0.0038778112682587526),
 ('VX', 0.008626399366751207)]

In [135]:
# Question 7: Find the largest departure delay for each carrier
LargestDepartureDelay_carrier = flights.map(lambda x: (x[4],float(x[11]))).reduceByKey(max)
LargestDepartureDelay_carrier.collect()

[('AA', 1988.0),
 ('NK', 836.0),
 ('UA', 1314.0),
 ('B6', 1006.0),
 ('MQ', 1544.0),
 ('F9', 1112.0),
 ('WN', 665.0),
 ('OO', 1378.0),
 ('EV', 1274.0),
 ('US', 759.0),
 ('DL', 1289.0),
 ('HA', 1433.0),
 ('AS', 963.0),
 ('VX', 644.0)]

In [136]:
# Question 8: Find the largest departure delay for each carrier for each month
LargestDepartureDelay_carrier_month = flights.map(lambda x: ((x[4],x[1]),float(x[11]))).reduceByKey(max)
LargestDepartureDelay_carrier_month.collect()

[(('US', '1'), 638.0),
 (('HA', '1'), 1003.0),
 (('F9', '1'), 696.0),
 (('AA', '2'), 1587.0),
 (('AS', '2'), 735.0),
 (('NK', '2'), 546.0),
 (('MQ', '2'), 1154.0),
 (('HA', '3'), 952.0),
 (('F9', '3'), 1112.0),
 (('MQ', '3'), 830.0),
 (('NK', '4'), 836.0),
 (('B6', '4'), 500.0),
 (('OO', '4'), 878.0),
 (('US', '5'), 711.0),
 (('HA', '5'), 326.0),
 (('B6', '5'), 544.0),
 (('EV', '5'), 730.0),
 (('MQ', '5'), 956.0),
 (('AS', '7'), 830.0),
 (('HA', '7'), 787.0),
 (('F9', '7'), 793.0),
 (('DL', '8'), 1207.0),
 (('NK', '9'), 693.0),
 (('AA', '9'), 1670.0),
 (('VX', '9'), 331.0),
 (('WN', '9'), 540.0),
 (('AA', '10'), 1573.0),
 (('AS', '10'), 580.0),
 (('HA', '10'), 1022.0),
 (('OO', '10'), 1122.0),
 (('EV', '10'), 975.0),
 (('NK', '11'), 476.0),
 (('NK', '12'), 723.0),
 (('EV', '12'), 1274.0),
 (('OO', '12'), 1378.0),
 (('WN', '12'), 562.0),
 (('VX', '12'), 363.0),
 (('AA', '1'), 1988.0),
 (('DL', '1'), 1184.0),
 (('NK', '1'), 557.0),
 (('VX', '1'), 397.0),
 (('DL', '2'), 1159.0),
 (('HA', 

In [137]:
# Question 9: For each carrier find the average departure delay.
TotalDepartureDelay_carrier = flights.map(lambda x: (x[4],float(x[11]))).reduceByKey(add)
cnt_carrier = flights.map(lambda x: (x[4],1)).reduceByKey(add)
TotalDepartureDelay_cnt = TotalDepartureDelay_carrier.leftOuterJoin(cnt_carrier)
AverageDepartureDelay_carrier = TotalDepartureDelay_cnt.map(lambda x: (x[0],(x[1][0],0)) if x[1][1] is None else (x[0],x[1])).map(lambda x: (x[0],x[1][0]/x[1][1]))
AverageDepartureDelay_carrier.collect()


[('AA', 8.773542906652084),
 ('NK', 15.68327383944317),
 ('UA', 14.26220664969373),
 ('B6', 11.333044995656211),
 ('MQ', 9.63204268375465),
 ('F9', 13.27060856928971),
 ('WN', 10.450107183471951),
 ('OO', 7.67823058605973),
 ('EV', 8.49219111957299),
 ('US', 6.0209194071912036),
 ('DL', 7.338090448359994),
 ('HA', 0.48473882945248586),
 ('AS', 1.7795501970785996),
 ('VX', 8.947094648078446)]

In [138]:
# Question 10: For each carrier find the average departure delay for each month.
TotalDepartureDelay_carrier_month = flights.map(lambda x: ((x[4],x[1]),float(x[11]))).reduceByKey(add)
cnt_carrier_month = flights.map(lambda x: ((x[4],x[1]),1)).reduceByKey(add)
TotalDepartureDelay_cnt_month = TotalDepartureDelay_carrier_month.leftOuterJoin(cnt_carrier_month)
AverageDepartureDelay_carrier_month = TotalDepartureDelay_cnt_month.map(lambda x: (x[0],(x[1][0],0)) if x[1][1] is None else (x[0],x[1])).map(lambda x: (x[0],x[1][0]/x[1][1]))
AverageDepartureDelay_carrier_month.collect()


[(('US', '1'), 5.051897638030398),
 (('HA', '1'), 1.3055900621118013),
 (('F9', '1'), 17.763215697759556),
 (('AA', '2'), 9.613731643027489),
 (('AS', '2'), 2.68008856814827),
 (('NK', '2'), 16.419458523921374),
 (('MQ', '2'), 15.991388270230141),
 (('HA', '3'), 0.8225883098368446),
 (('F9', '3'), 19.849928057553957),
 (('MQ', '3'), 11.44983301357209),
 (('NK', '4'), 9.86573294018534),
 (('B6', '4'), 8.2763851044505),
 (('OO', '4'), 5.276855399460763),
 (('US', '5'), 4.043807944077486),
 (('HA', '5'), -1.3484612993472178),
 (('B6', '5'), 6.981520053179703),
 (('EV', '5'), 9.886026050027432),
 (('MQ', '5'), 9.21988911171405),
 (('AS', '7'), 2.5547689779407117),
 (('HA', '7'), 0.7089863407620417),
 (('F9', '7'), 13.785784919653894),
 (('DL', '8'), 7.74069452851866),
 (('NK', '9'), 9.6110776035384),
 (('AA', '9'), 4.9449297482931085),
 (('VX', '9'), 3.2865735351183547),
 (('WN', '9'), 5.360186795171146),
 (('AA', '10'), 5.265661793246216),
 (('AS', '10'), -0.08820073270201148),
 (('HA', '

In [140]:
# Question 11: Which date of the year has the highest rate of flight cancellations?
# You should calculate the rate of flight cancellation by dividing number of canceled flights by total
# number of flights.
total_flight_date = flights.map(lambda x: ((x[0],x[1],x[2]),1)).reduceByKey(add)
cancelled_flight_date = flights.filter(lambda x: x[12]=="1").map(lambda x: ((x[0],x[1],x[2]),1)).reduceByKey(add)

total_flight_date_cancelled_flight_date = total_flight_date.leftOuterJoin(cancelled_flight_date)
DateOfYear_cancellations = total_flight_date_cancelled_flight_date.map(lambda x: (x[0],(x[1][0],0)) if x[1][1] is None else (x[0],x[1])).map(lambda x: (x[0],x[1][1]/x[1][0]))

DateOfYear_cancellations.top(1, lambda x:x[1])

[(('2015', '1', '27'), 0.19030023094688223)]

In [141]:
# Question 12: Calculate the number of flights to each destination state for each carrier, for which
# state do they have the largest average delay? You will need the airline and airport data sets for this
# question.
destination_airline = flights.map(lambda x: (x[8],x[4]))
airport_state = airports.map(lambda x: (x[0],x[3]))
destination_airline_state = destination_airline.leftOuterJoin(airport_state)
state_airline = destination_airline_state.map(lambda x: ((x[1][1],x[1][0]),1)).reduceByKey(add)
state_airline.collect()

destination_cnt = flights.map(lambda x: (x[8],1))
destination_state_cnt = destination_cnt.join(airport_state)

state_cnt = destination_state_cnt.map(lambda x: (x[1][1],1)).reduceByKey(add)
delayed_destination_cnt = flights.filter(lambda x: x[11] == "1").map(lambda x: (x[8],1))
delayed_destination_state_cnt = delayed_destination_cnt.join(airport_state)
state_DelayCnt = delayed_destination_state_cnt.map(lambda x: (x[1][1],1)).reduceByKey(add)

state_delay = state_cnt.join(state_DelayCnt)
state_AvgDelay = state_delay.map(lambda x: (x[0],x[1][1]/x[1][0]))
state_AvgDelay = state_AvgDelay.map(lambda x: (x[0],0) if x[1] is None else (x[0],x[1]))


largest_state_AvgDelay = state_AvgDelay.top(1, lambda x: x[1])
largest_state_AvgDelay


[('GU', 0.03592814371257485)]

In [ ]:
sc.stop()